In [0]:
# Instalamos KERAS
!pip install -q keras

In [2]:
# Descargamos el dataset
!git clone https://github.com/agonzalezhidalgo/dataset_detector.git

Cloning into 'dataset_detector'...
remote: Enumerating objects: 29652, done.
remote: Counting objects: 100% (29652/29652), done.
remote: Compressing objects: 100% (28571/28571), done.
remote: Total 29652 (delta 1082), reused 29649 (delta 1081), pack-reused 0
Receiving objects: 100% (29652/29652), 877.02 MiB | 30.68 MiB/s, done.
Resolving deltas: 100% (1082/1082), done.
Checking out files: 100% (30510/30510), done.


In [3]:
# http://scikit-image.org/docs/stable/api/api.html
import skimage

# https://docs.python.org/3/library/csv.html
import csv

# https://matplotlib.org/api/index.html
import matplotlib

# https://matplotlib.org/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot
import matplotlib.pyplot as plt

# https://docs.python.org/3/library/os.html
import os

# https://docs.scipy.org/doc/numpy/reference/
import numpy as np

# https://keras.io/models/model/
import keras

from keras import models

# Core Layers: https://keras.io/layers/core/
# Convolution Layers: https://keras.io/layers/convolutional/
from keras import layers

# https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

# https://keras.io/callbacks/
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


Celda con funciones generales

In [0]:
# Devuelve las imagenes y etiquetas de una carpeta especifíca
def __read_images_labels_from_dir(directory, images, labels, shape, as_gray, get_label_from_dir):
    
    #Buscamos fotos en el directorio
    for f in os.listdir(directory):
        # Cargamos archivos con extension .ppm 
        # Obviamos el color y lo cargamos como escala de grises y normalizamos el tamaño
        if f.endswith(".ppm"):
            image = load_image(os.path.join(directory, f), shape, as_gray)
            images.append(image)
            if get_label_from_dir:
                labels.append(int(os.path.basename(directory)))
    
    return images, labels
  
# Obtenemos el índice inicial y final de una label específica
# - label: índice de la label que se quiere buscar.
# - source: lista de la relación de etiquetas.

def __get_start_end(label, source):

    start = 0
    end = 0
    
    try:        
        start = source.index(label)
        end = start + source.count(label)

    except:
        print("label doesn't exist")

    return start, end
  
# Devuelve una colección de directorios de la ruta
def __get_directories(data_dir):
    
    directories = []
    
    if os.path.exists(data_dir):
        
        #Buscamos todos los directorios de la ruta
        for d in os.listdir(data_dir):
            if os.path.isdir(os.path.join(data_dir, d)):
                directories.append(d)
                
    else:
        print("path doesn't exists")
    return directories
  
# Devuelve una colección con las imágenes y los labels de la ruta
# -datadir: path donde se encuentran la colección de imágenes.
# -shape: Dimensiones con las que se cargarán las imágenes.
# -as_gray: Indica si la imagen se cargará en escala de grises.
# -get_label_from_dir: (defecto True) específica si la categoría se lee del propio directorio
def readDataset(data_dir, shape, as_gray, get_label_from_dir = True):
    images = []
    labels = []
    directories = __get_directories(data_dir)
    
    images, labels = __read_images_labels_from_dir(data_dir, images, labels, shape, as_gray, get_label_from_dir)
    
    for d in directories:
                
        #Buscamos fotos en el directorio
        images_dir = os.path.join(data_dir, d)
        images, labels = __read_images_labels_from_dir(images_dir, images, labels, shape, as_gray, get_label_from_dir)
       
    return images, labels
  
# Devuelve una imagen
# - path: ruta de la imagen.
# - size: dimensión con la que se cargará la imagen.
# - as_gray: True para cargar la imagen en escala de grises.

def load_image(path, size, as_gray):
    aux = skimage.data.imread(path, as_gray = as_gray)
    return skimage.transform.resize(aux, size, mode='constant')
  
# Devuelve una lista con los datos del fichero csv.
# - path: ruta hasta el fichero csv.
# - delimeter: Carácter delimitador de campos

def read_csv(path, delimiter):
    file = open(path)
    title_csv = csv.reader(file, delimiter = delimiter)
    return list(title_csv)
  
# Imprime los tamaños de las colecciones
# - images: lista de imágenes precargadas.
# - labels: relación de las imágenes con las categorías a las que pertenecen.
# - np_images: np.array con las imágenes precargadas.
# - np_labels: np.array con las categorías precargadas.
# - environment: string identificativo del entorno.

def print_size_dataset(images, labels, np_images, np_labels, environment):
    print("Total images (" + environment + "): ", len(images))
    print("Total labels (" + environment + "): ", len(set(labels)))
    print("Images shape: ", np_images.shape)
    print("Labels shape: ", np_labels.shape)
    
# Imprime una matriz 32x32 con los diferentes tipos de señales que
# se van a clasificar. Muestra la primera imagen de cada categoría.
# - images: lista de imágenes precargadas.
# - labels: relación de las imágenes con las categorías a las que pertenecen.
# - titles: lista con los nombres de las categorías.

def print_summary_dataset(images, labels, titles):
    
    #Quitamos los repetidos a los labels
    unique_labels = set(labels)
    
    plt.figure(figsize=(32, 32))
    i = 1
    
    for label in unique_labels:
        
        #Obtenemos la primera imagén de cada label
        image = images[labels.index(label)]
        plt.subplot(8, 8, i)
        plt.axis('off')
        plt.title(titles[i-1][1])
        i += 1
        _ = plt.imshow(image)
        
    plt.show()

# Imprime todas las imágenes de una label especifica
# - label: índice de la label que se quiere buscar.
# - images: lista de imágenes precargadas.
# - source: lista de la relación de etiquetas.
# - titles: lista con las diferentes categorías de labels.

def print_signals(label, images, source, titles):
    
    start, end = __get_start_end(label, source)
    
    if start < end:
        rows = (end - start) / 8

        print("Signal: ", titles[label][1])
        plt.figure(figsize=(10, 10))
        i = 1

        for image in images[start:end]:
            plt.subplot(rows + 1, 8, i)
            plt.axis('off')
            i += 1
            plt.imshow(image)

        plt.show()
        
# Imprime los atributos de las imágenes de una label especifica.
# - label: índice de la label que se quiere buscar.
# - images: lista de imágenes precargadas.
# - source: lista de la relación de etiquetas.
# - titles: lista con las diferentes categorías de labels.

def print_signals_attributes(label, images, source, titles):
    
    start, end = __get_start_end(label, source)
    
    if start < end:
        print("Signal: ", titles[label][1])
        for image in images[start:end]:
            print("shape: ", image.shape, "\tmin:", image.min(), "\tmax: ", image.max())
            
# Devuelve una lista con las categorías de las imágenes de prueba leídas del fichero csv.
# - csv: fichero csv
# - class_column: número de columna que contiene las clases.
# - first_is_header: Indica si la primera fila es una cabecerá.

def get_class_id_array(csv, class_column, first_is_header = True):
    labels = []
    for row in csv:
        if not first_is_header:
            labels.append(int(row[class_column]))
        else:
            first_is_header = False
    
    return labels

In [5]:
# Mostramos información de CPU & GPU

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1731755466473237525
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9829624130609613921
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10969816919185812521
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14892338381
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14136930018601045405
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [6]:
# Obtenemos el directorio actual como trabajo.
ROOT_PATH = os.getcwd()

# Establecemos la dimensión de las imágenes.
IMG_SHAPE = (64, 64)
print("Tamaño de las imágenes de entrada: ", IMG_SHAPE)
IMG_SHAPE_LEN = IMG_SHAPE[0] * IMG_SHAPE[1]
print("Vectorizando la entrada, sería de un tamaño: ", IMG_SHAPE_LEN)

# Obtenemos los paths de trabajo
train_path = os.path.join(ROOT_PATH, "dataset_detector/train")
test_path = os.path.join(ROOT_PATH, "dataset_detector/test")

Tamaño de las imágenes de entrada:  (64, 64)
Vectorizando la entrada, sería de un tamaño:  4096


In [7]:
# Cargamos las imágenes de entrenamiento y de test.
images_train, labels_train = readDataset(train_path, IMG_SHAPE, False)
images_test, labels_test = readDataset(test_path, IMG_SHAPE, False)

# Convertimos las listas a array numpy de float32
np_images_train = np.asarray(images_train, dtype = np.float32)
np_labels_train = np.asarray(labels_train, dtype = np.int8)

np_images_test = np.asarray(images_test, dtype = np.float32)
np_labels_test = np.asarray(labels_test, dtype = np.int8)


# Se imprime información de los datos cargados.
print_size_dataset(images_train, labels_train, np_images_train, np_labels_train, "train")
print_size_dataset(images_test, labels_test, np_images_test, np_labels_test, "test")

Total images (train):  15481
Total labels (train):  2
Images shape:  (15481, 64, 64, 3)
Labels shape:  (15481,)
Total images (test):  14726
Total labels (test):  2
Images shape:  (14726, 64, 64, 3)
Labels shape:  (14726,)


In [8]:
# Convertimos las labels de manera categórica
labels_categorical_train = keras.utils.to_categorical(np_labels_train)
labels_categorical_test = keras.utils.to_categorical(np_labels_test)

print("Ejemplo primera imagen de manera categórica: ", labels_categorical_train[0])

Ejemplo primera imagen de manera categórica:  [1. 0.]


In [0]:
def get_keras_model():
    # IMPLEMENTACIÓN RED NEURONAL
    # En Keras la envoltura para cualquier red neuronal se crea con la clase Sequential
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5, 5),
                            activation='elu', input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, (5, 5), activation='elu'))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Conv2D(128, (5, 5), activation='elu'))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(len(set(labels_train)), activation='softmax'))
    return model

In [10]:
model = get_keras_model()

# Muestra la arquitectura de nuestra red neuronal
model.summary()

# Configurando el modelo de aprendijaze:
#  · loss, función para evaluar el grado de error entre salidas calculadas
#  · optimizador, función para calcular los pesos de los parámetros a partir de los datos de entrada
#  · metricas, para monitorizar el proceso de aprendizaje de la red.
model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

# Entrenamiento del modelo
# - batch_size, indica el número de datos que se usan en cada actualización.
# - epochs, indica el número de veces que se usan todos los datos del proceso.
#model.fit(np_images_train, labels_categorical_train, batch_size=32, epochs=20)
model.fit(np_images_train, labels_categorical_train,
          batch_size = 64,
          epochs = 25,
          verbose = 0,
          callbacks=[ModelCheckpoint('model_64_64_gpu_detector.h5', save_best_only = False)])

# Evaluación del modelo
test_loss, test_acc = model.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

W0616 10:57:18.848674 139743172458368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0616 10:57:18.859019 139743172458368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0616 10:57:18.869326 139743172458368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0616 10:57:18.894334 139743172458368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0616 10:57:18.897216 139743172458368 deprecation_wrapp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         204928    
__________

In [0]:
datagen = ImageDataGenerator(featurewise_center=False,
                             featurewise_std_normalization=False, 
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                             shear_range=0.1,
                             rotation_range=10.,)

datagen.fit(np_images_test)

In [12]:
model_2 = get_keras_model()

model_2.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model_2.fit_generator(datagen.flow(np_images_train, labels_categorical_train, batch_size=32),
                      steps_per_epoch = np_images_train.shape[0],
                      epochs = 25,
                      callbacks=[ModelCheckpoint('model_data_aug_64_64_gpu_detector.h5',save_best_only = False)])

Epoch 1/25
15481/15481 [==============================] - 568s 37ms/step - loss: 0.0509 - acc: 0.9869
Epoch 2/25
15481/15481 [==============================] - 564s 36ms/step - loss: 0.0614 - acc: 0.9928
Epoch 3/25
15481/15481 [==============================] - 576s 37ms/step - loss: 0.1392 - acc: 0.9905
Epoch 4/25
15481/15481 [==============================] - 573s 37ms/step - loss: 0.3367 - acc: 0.9788
Epoch 5/25
15481/15481 [==============================] - 597s 39ms/step - loss: 0.3665 - acc: 0.9771
Epoch 6/25
15481/15481 [==============================] - 575s 37ms/step - loss: 0.4190 - acc: 0.9739
Epoch 7/25
15481/15481 [==============================] - 575s 37ms/step - loss: 0.4140 - acc: 0.9743
Epoch 8/25
15481/15481 [==============================] - 588s 38ms/step - loss: 0.7588 - acc: 0.9529
Epoch 9/25
15481/15481 [==============================] - 579s 37ms/step - loss: 0.8306 - acc: 0.9484
Epoch 10/25
15481/15481 [==============================] - 594s 38ms/step - loss: 

In [13]:
# Evaluación del modelo
test_loss, test_acc = model_2.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

14726/14726 [==============================] - 2s 119us/step
Test loss: 2.3941728629674763
Test accuracy: 0.8514192584463391
